In [6]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [5]:
# @title 📦 Install deps (Colab)
!pip -q install pillow numpy imageio imageio-ffmpeg simplekml ipywidgets > /dev/null
from google.colab import output
output.enable_custom_widget_manager()
print("Deps ready.")

Deps ready.


In [7]:
# @title 🔧 Load core (CRC + nested + header pass + depth-change) — v5.1
import importlib.util, sys, os
core_path = "/content/rsd_core_crc.py"
with open(core_path,"w",encoding="utf-8") as f: f.write("\n# rsd_core_crc.py  \u2014 spec-aligned Garmin RSD parser + pipeline (CRC, nested, header pass)\nimport os, math, mmap, struct, zipfile, logging, zlib, json\nfrom dataclasses import dataclass\nfrom typing import Optional, List, Tuple\nimport numpy as np\nfrom PIL import Image\nimport imageio.v2 as imageio\n\nlogging.getLogger().setLevel(logging.INFO)\nEARTH_R = 6371000.0\n\ndef meters_to_deg_lat(dy): return (dy / EARTH_R) * (180.0 / math.pi)\ndef meters_to_deg_lon(dx, lat): return (dx / (EARTH_R * math.cos(math.radians(lat)))) * (180.0 / math.pi)\ndef offset_latlon(lat, lon, dx_east_m, dy_north_m):\n    return lat + meters_to_deg_lat(dy_north_m), lon + meters_to_deg_lon(dx_east_m, lat)\n\ndef read_varuint_from(mm, pos):\n    res=0; shift=0; size=len(mm)\n    while pos < size:\n        b=mm[pos]; pos+=1\n        res |= (b & 0x7F) << shift\n        if not (b & 0x80): break\n        shift+=7\n    return res, pos\n\ndef zigzag_to_int32(u): return (u>>1) ^ (-(u & 1))\n\ndef crc32_ieee(data: bytes) -> int:\n    return (zlib.crc32(data) ^ 0xFFFFFFFF) & 0xFFFFFFFF\n\ndef _read_varstruct(mm, pos, size, *, validate_crc: bool=True):\n    start = pos\n    n, pos = read_varuint_from(mm, pos)\n    fields = []\n    for _ in range(n):\n        key, pos = read_varuint_from(mm, pos)\n        flen = key & 7\n        field_no = key >> 3\n        if flen == 7:\n            vlen, pos = read_varuint_from(mm, pos)\n        else:\n            vlen = flen\n        if pos+vlen > size: raise ValueError(\"Varstruct value exceeds file size\")\n        val = bytes(mm[pos:pos+vlen]); pos += vlen\n        fields.append((field_no, val))\n    if pos + 4 > size: raise ValueError(\"Varstruct truncated before CRC\")\n    crc_read = struct.unpack('<I', mm[pos:pos+4])[0]\n    if validate_crc:\n        crc_calc = crc32_ieee(bytes(mm[start:pos]))\n        if crc_calc != crc_read:\n            raise ValueError(f\"CRC mismatch: calc=0x{crc_calc:08X} read=0x{crc_read:08X} at 0x{start:X}\")\n    pos += 4\n    return fields, pos\n\n@dataclass\nclass RSDRecord:\n    offset:int\n    channel_id:Optional[int]\n    sequence_count:int\n    data_size:int\n    rec_time_ms:int\n    lat_deg:Optional[float]\n    lon_deg:Optional[float]\n    water_temp_c:Optional[float]\n    bottom_depth_m:Optional[float]\n    sample_cnt:Optional[int]\n    sonar_data_offset:Optional[int]\n    sonar_data_size:Optional[int]\n    first_sample_depth_m:Optional[float]\n    last_sample_depth_m:Optional[float]\n\nclass RSDParser:\n    MAGIC_REC_HDR=0xB7E9DA86; MAGIC_REC_TRL=0xF98EACBC; HEADER_AREA_END=0x5000\n    @staticmethod\n    def mapunit_to_deg(x:int)->float: return x * (360.0 / float(1<<32))\n    def __init__(self, path):\n        self.path=path; self.size=os.path.getsize(path); self.header_items=[]\n    def __enter__(self):\n        self.f=open(self.path,'rb'); self.mm=mmap.mmap(self.f.fileno(),0,access=mmap.ACCESS_READ)\n        self.header_items = self.parse_header_area()\n        return self\n    def __exit__(self, a,b,c):\n        try:self.mm.close()\n        finally:self.f.close()\n    def parse_header_area(self):\n        mv = memoryview(self.mm)\n        pos = 0; items=[]\n        while pos < self.HEADER_AREA_END:\n            try:\n                fields, new_pos = _read_varstruct(mv, pos, self.size)\n            except Exception:\n                break\n            items.append((pos, fields))\n            if new_pos <= pos: break\n            pos = new_pos\n        logging.info(\"Header items parsed: %d\", len(items))\n        return items\n    def parse_records(self):\n        mv = memoryview(self.mm)\n        pos = self.HEADER_AREA_END\n        rec_idx=0\n        while pos + 12 <= self.size:\n            rec_start = pos\n            try:\n                fields, pos_after_hdr = _read_varstruct(mv, pos, self.size)\n            except Exception as e:\n                logging.error(\"Header parse error at 0x%X: %s\", pos, e)\n                pos = rec_start + 4; continue\n            magic=None; seq=0; data_size=0; rec_time_ms=0\n            for fn,val in fields:\n                if fn==0 and len(val)==4: magic=struct.unpack('<I', val)[0]\n                elif fn==2 and len(val)==4: seq=struct.unpack('<I', val)[0]\n                elif fn==4 and len(val)==2: data_size=struct.unpack('<H', val)[0]\n                elif fn==5 and len(val)==4: rec_time_ms=struct.unpack('<I', val)[0]\n            if magic != self.MAGIC_REC_HDR:\n                pos = rec_start + 4; continue\n            # Body\n            body_start = pos_after_hdr\n            channel_id=None; lat=None; lon=None; wt=None; bd=None; sc=None; fsd=None; lsd=None\n            sonar_ofs=None; sonar_size=None\n            if data_size>0:\n                try:\n                    b_fields, body_end = _read_varstruct(mv, body_start, self.size)\n                except Exception as e:\n                    logging.error(\"Body parse error (rec %d at 0x%X): %s\", rec_idx, body_start, e)\n                    pos = rec_start + 4; continue\n                def _dec_varint(b):\n                    v=0;shift=0\n                    for bb in b:\n                        v |= (bb & 0x7F) << shift\n                        if not (bb & 0x80): break\n                        shift+=7\n                    return v\n                for fn,val in b_fields:\n                    if fn==0: channel_id=int(_dec_varint(val))\n                    elif fn==1: bd = zigzag_to_int32(_dec_varint(val))/1000.0\n                    elif fn==3: fsd = zigzag_to_int32(_dec_varint(val))/1000.0\n                    elif fn==4: lsd = zigzag_to_int32(_dec_varint(val))/1000.0\n                    elif fn==7 and len(val)==4: sc = struct.unpack('<I', val)[0]\n                    elif fn==9 and len(val)==4: lat = self.mapunit_to_deg(struct.unpack('<i', val)[0])\n                    elif fn==10 and len(val)==4: lon = self.mapunit_to_deg(struct.unpack('<i', val)[0])\n                    elif fn==11 and len(val)==4: wt = struct.unpack('<f', val)[0]\n                used = body_end - body_start\n                sonar_ofs = body_end; sonar_size = max(0, data_size - used)\n                pos = body_start + data_size\n            else:\n                pos = body_start\n            # Trailer\n            try:\n                if pos + 12 > self.size: break\n                tr_magic, chunk_size, tr_crc = struct.unpack('<III', self.mm[pos:pos+12])\n                if tr_magic != self.MAGIC_REC_TRL or chunk_size <= 0:\n                    raise ValueError(f\"Bad trailer magic/size at 0x{pos:X}\")\n                pos = rec_start + chunk_size\n            except Exception as e:\n                logging.error(\"Trailer error (rec %d at 0x%X): %s\", rec_idx, pos, e)\n                pos = rec_start + 4; continue\n            yield RSDRecord(rec_start, channel_id, seq, data_size, rec_time_ms, lat, lon, wt, bd, sc,\n                            sonar_ofs, sonar_size, fsd, lsd)\n            rec_idx += 1\n\n# ---------- Rendering / tones ----------\ndef tone_map(u8, invert=True, lo_pct=1.0, hi_pct=99.0, gamma=1.0):\n    a=u8.astype(np.float32)\n    if invert: a=255.0 - a\n    lo=np.percentile(a, lo_pct) if 0 <= lo_pct < 50 else a.min()\n    hi=np.percentile(a, hi_pct) if 50 < hi_pct <= 100 else a.max()\n    if hi<=lo: hi=lo+1.0\n    a=(a-lo)/(hi-lo)\n    if gamma!=1.0: a = np.power(np.clip(a,0,1),gamma)\n    return np.clip(a*255.0,0,255).astype(np.uint8)\n\ndef make_palette(name:str):\n    import numpy as _np\n    x = _np.linspace(0,1,256)\n    if name==\"amber\":\n        r=_np.clip(3.0*x,0,1); g=_np.clip(1.8*x,0,1); b=_np.clip(0.5*x,0,1)\n    elif name==\"blue\":\n        r=_np.clip(0.4*x,0,1); g=_np.clip(0.7*x,0,1); b=_np.clip(1.8*x,0,1)\n    elif name==\"green\":\n        r=_np.clip(0.5*x,0,1); g=_np.clip(1.8*x,0,1); b=_np.clip(0.5*x,0,1)\n    elif name==\"ironbow\":\n        r=_np.clip(1.5*x,0,1); g=_np.clip(1.5*_np.maximum(x-0.33,0),0,1); b=_np.clip(1.5*_np.maximum(x-0.66,0),0,1)\n    else:\n        r=g=b=x\n    return ( _np.stack([r,g,b],1)*255.0 + 0.5 ).astype(_np.uint8)\n\ndef apply_palette(u8row, lut):\n    if u8row.ndim==1: u8row=u8row[np.newaxis,:]\n    return lut[u8row]\n\ndef slant_resample_half(row_u8, first_sd_m, last_sd_m):\n    if first_sd_m is None or last_sd_m is None: return row_u8\n    sc=row_u8.size\n    d = np.linspace(max(0.0, first_sd_m), max(first_sd_m, last_sd_m), sc)\n    s = np.linspace(d[0], d[-1] + (d[-1]-d[0]) * 0.25, sc)\n    x = np.sqrt(np.maximum(s*s - np.maximum(d,1e-3)**2, 0.0))\n    x_norm = (x - x.min()) / max(1e-6, (x.max()-x.min()))\n    xi = np.linspace(0,1,sc)\n    idx = np.clip((np.interp(xi, x_norm, np.arange(sc))).round().astype(int), 0, sc-1)\n    return row_u8[idx]\n\ndef infer_layout(blob_len, sample_cnt):\n    sc=int(sample_cnt or 0)\n    if sc>0:\n        if blob_len == sc: return ('u8',1)\n        if blob_len == 2*sc: return ('u8',2)\n        if blob_len == 4*sc: return ('u16',2)\n        if blob_len == 2*sc: return ('u16',1)\n        ratio = blob_len / sc\n        if abs(ratio-2.0)<0.06: return ('u8',2)\n        if abs(ratio-1.0)<0.1: return ('u8',1)\n        if abs(ratio-4.0)<0.2: return ('u16',2)\n        if abs(ratio-2.0)<0.2: return ('u16',1)\n    return ('u8',2)\n\n# ---------- KMZ helpers ----------\ndef segment_quad(latA, lonA, latB, lonB, half_width_m):\n    dN = (latB - latA) * (math.pi/180.0) * EARTH_R\n    dE = (lonB - lonA) * (math.pi/180.0) * EARTH_R * math.cos(math.radians((latA+latB)/2.0))\n    L  = math.hypot(dE, dN)\n    if L == 0: dE, dN = 1.0, 0.0; L = 1.0\n    uE, uN = dE/L, dN/L\n    pE, pN = -uN, uE\n    A_left  = offset_latlon(latA, lonA,  pE*half_width_m,  pN*half_width_m)\n    A_right = offset_latlon(latA, lonA, -pE*half_width_m, -pN*half_width_m)\n    B_left  = offset_latlon(latB, lonB,  pE*half_width_m,  pN*half_width_m)\n    B_right = offset_latlon(latB, lonB, -pE*half_width_m, -pN*half_width_m)\n    return [(A_left[1],A_left[0]), (A_right[1],A_right[0]), (B_right[1],B_right[0]), (B_left[1],B_left[0])]\n\n# ---------- Pipeline entry ----------\ndef build_rows_and_assets(rsd_path, out_dir, cfg):\n    os.makedirs(out_dir, exist_ok=True)\n    # defaults\n    ROW_H = int(cfg.get(\"ROW_HEIGHT_PX\", 40))\n    WATER_PX = int(cfg.get(\"WATER_COLUMN_PX\", 8))\n    INVERT = bool(cfg.get(\"INVERT\", True))\n    CL = float(cfg.get(\"CLIP_LOW_PCT\", 1.0))\n    CH = float(cfg.get(\"CLIP_HIGH_PCT\", 99.0))\n    GM = float(cfg.get(\"GAMMA\", 1.0))\n    PALETTES = list(cfg.get(\"PALETTES\", [\"amber\",\"grayscale\"]))\n    APPLY_SLANT = bool(cfg.get(\"APPLY_SLANT\", True))\n    STRIDE = int(cfg.get(\"STRIDE\", 2))\n    SWATH_SS = float(cfg.get(\"SWATH_M_SS\", 40.0))\n    MAKE_BUCKETED = bool(cfg.get(\"MAKE_BUCKETED_KMZ\", True))\n    MAKE_REGION = bool(cfg.get(\"MAKE_REGIONATED_KMZ\", True))\n    MAKE_TARGETS = bool(cfg.get(\"MAKE_TARGETS\", True))\n    TARGET_PCT = float(cfg.get(\"TARGET_PCT\", 99.5))\n    TARGET_MIN_SPAN = int(cfg.get(\"TARGET_MIN_SPAN\", 3))\n    MAKE_DEPTH_GPS = bool(cfg.get(\"MAKE_DEPTH_GPS\", True))\n    GPS_STEP_M = float(cfg.get(\"GPS_STEP_M\", 100.0))\n    DEPTH_EVERY = int(cfg.get(\"DEPTH_EVERY_PINGS\", 20))\n    MAKE_VIDEO = bool(cfg.get(\"MAKE_VIDEO\", False))\n    VIDEO_FPS = int(cfg.get(\"VIDEO_FPS\", 30))\n    VIDEO_H = int(cfg.get(\"VIDEO_HEIGHT\", 1080))\n    VIDEO_MAX = int(cfg.get(\"VIDEO_MAX_FRAMES\", 20000))\n    PREVIEW_MAX = int(cfg.get(\"PREVIEW_MAX_ROWS\", 2000))\n    # Depth-change thresholds\n    DEP_FT_THRESHOLD = float(cfg.get(\"DEP_FT_THRESHOLD\", 3.0))\n    PCT_CHANGE_MIN = cfg.get(\"PCT_CHANGE_MIN\", None)\n    if isinstance(PCT_CHANGE_MIN, str) and PCT_CHANGE_MIN.strip()==\"\":\n        PCT_CHANGE_MIN = None\n    if PCT_CHANGE_MIN is not None: PCT_CHANGE_MIN = float(PCT_CHANGE_MIN)\n    DEP_M_THRESHOLD = DEP_FT_THRESHOLD * 0.3048\n\n    base = os.path.splitext(os.path.basename(rsd_path))[0]\n    # parse & render strips\n    strip_paths=[]; latlons=[]; depths=[]; preview_rows=[]\n    with RSDParser(rsd_path) as P, open(rsd_path,'rb') as f:\n        for i, r in enumerate(P.parse_records()):\n            if r.sonar_data_offset is None or (r.sonar_data_size or 0) <= 0:\n                continue\n            if (i % STRIDE) != 0: continue\n            f.seek(r.sonar_data_offset); blob=f.read(r.sonar_data_size)\n            dtype,chans = infer_layout(len(blob), r.sample_cnt)\n            a=np.frombuffer(blob, dtype=np.uint8 if dtype=='u8' else '<u2')\n            if dtype!='u8':\n                a=(a.astype(np.float32)*(255.0/65535.0)).clip(0,255).astype(np.uint8)\n            sc=int(r.sample_cnt or 0)\n            if sc<=0: sc=a.size//max(1,chans)\n            if chans>=2 and a.size >= 2*sc:\n                port=a[:sc][::-1]; stbd=a[sc:2*sc]\n            else:\n                half=a.size//2; port=a[:half][::-1]; stbd=a[half:half+half]\n            if APPLY_SLANT:\n                port=slant_resample_half(port, r.first_sample_depth_m, r.last_sample_depth_m)\n                stbd=slant_resample_half(stbd, r.first_sample_depth_m, r.last_sample_depth_m)\n            scan=np.hstack([port, np.zeros(WATER_PX, dtype=np.uint8), stbd])\n            g=tone_map(scan, INVERT, CL, CH, GM)\n            img = Image.fromarray(g[np.newaxis,:],'L').resize((g.shape[0], ROW_H))\n            png = f\"{base}_row_{i:06d}.png\"; img.save(os.path.join(out_dir, png))\n            strip_paths.append(png)\n            latlons.append((r.lon_deg, r.lat_deg))\n            depths.append(r.bottom_depth_m if r.bottom_depth_m is not None else 0.0)\n            if len(preview_rows) < PREVIEW_MAX:\n                preview_rows.append(np.array(img))\n    # preview\n    if preview_rows:\n        prev = np.vstack(preview_rows)\n        Image.fromarray(prev,'L').save(os.path.join(out_dir, f\"{base}_waterfall.png\"))\n    # recolors\n    for pal in PALETTES:\n        lut = make_palette(pal)\n        for png in strip_paths:\n            g = np.array(Image.open(os.path.join(out_dir, png)))\n            rgb = lut[g[:,0,:]] if g.ndim==3 else lut[g]\n            Image.fromarray(rgb.astype(np.uint8),'RGB').save(os.path.join(out_dir, png.replace(\".png\", f\"_{pal}.png\")))\n\n    # Depth+GPS and depth-change KMLs\n    import simplekml\n    if MAKE_DEPTH_GPS:\n        k = simplekml.Kml()\n        ls = k.newlinestring(name=f\"{base} track\")\n        coords=[]\n        for (lon,lat) in latlons:\n            if lon is None or lat is None: continue\n            coords.append((lon,lat,0))\n        ls.coords = coords\n        # GPS ticks every GPS_STEP_M\n        ticks = k.newfolder(name=\"GPS ticks\")\n        last = None\n        for (lon,lat) in latlons:\n            if lon is None or lat is None: continue\n            if last is None:\n                last=(lat,lon); ticks.newpoint(coords=[(lon,lat,0)])\n                continue\n            dN=(lat-last[0])*(math.pi/180.0)*EARTH_R\n            dE=(lon-last[1])*(math.pi/180.0)*EARTH_R*math.cos(math.radians((lat+last[0])/2.0))\n            if math.hypot(dE,dN)>=GPS_STEP_M:\n                ticks.newpoint(coords=[(lon,lat,0)]); last=(lat,lon)\n        k.save(os.path.join(out_dir, f\"{base}_track.kml\"))\n\n        # Depth markers every DEPTH_EVERY pings\n        kd = simplekml.Kml()\n        f_dep = kd.newfolder(name=\"Depth pings\")\n        for idx, ((lon,lat), d) in enumerate(zip(latlons, depths)):\n            if lon is None or lat is None: continue\n            if idx % max(1, DEPTH_EVERY) == 0:\n                f_dep.newpoint(name=f\"{d:.1f} m\", coords=[(lon,lat,0)])\n        kd.save(os.path.join(out_dir, f\"{base}_depth_every.kml\"))\n\n        # Depth-change markers (> ft or %)\n        kc = simplekml.Kml()\n        f_ch = kc.newfolder(name=\"Depth change\")\n        prev=None\n        for (lon,lat), d in zip(latlons, depths):\n            if lon is None or lat is None or d is None: continue\n            fire=False; label=None\n            if prev is not None:\n                if abs(d - prev) >= DEP_M_THRESHOLD:\n                    fire=True; label = f\"\u0394depth {d:.1f} m (> {DEP_FT_THRESHOLD:.1f} ft)\"\n                if PCT_CHANGE_MIN is not None and prev>0:\n                    pct = 100.0 * abs(d-prev)/prev\n                    if pct >= PCT_CHANGE_MIN:\n                        fire=True; label = f\"\u0394depth {d:.1f} m ({pct:.1f}%)\"\n            if fire:\n                f_ch.newpoint(name=label or f\"{d:.1f} m\", coords=[(lon,lat,0)])\n            prev = d\n        kc.save(os.path.join(out_dir, f\"{base}_depth_change.kml\"))\n\n    # KMZ writers (bucketed + regionated)\n    def write_bucketed_kmz(palette_suffix=\"\"):\n        kmz_path = os.path.join(out_dir, f\"{base}_sidescan_bucketed{palette_suffix}.kmz\")\n        images_dir_in_kmz = \"files\"\n        kml_lines = ['<?xml version=\"1.0\" encoding=\"UTF-8\"?>',\n                     '<kml xmlns=\"http://www.opengis.net/kml/2.2\" xmlns:gx=\"http://www.google.com/kml/ext/2.2\">',\n                     '<Document>',\n                     f'<name>{base} sidescan (bucketed{palette_suffix})</name>']\n        used = []\n        for i in range(0, len(strip_paths)-1):\n            img = strip_paths[i].replace(\".png\", f\"{palette_suffix}.png\") if palette_suffix else strip_paths[i]\n            lonA,latA = latlons[i]\n            lonB,latB = latlons[i+1] if i+1 < len(latlons) else latlons[i]\n            if None in (lonA,latA,lonB,latB): continue\n            quad = segment_quad(latA, lonA, latB, lonB, SWATH_SS)\n            coords_txt = \" \".join([f\"{lon:.8f},{lat:.8f}\" for lon,lat in quad])\n            kml_lines += ['<GroundOverlay>',\n                          f'  <name>{img}</name>',\n                          '  <Icon>',\n                          f'    <href>{images_dir_in_kmz}/{img}</href>',\n                          '  </Icon>',\n                          '  <gx:LatLonQuad>',\n                          f'    <coordinates>{coords_txt}</coordinates>',\n                          '  </gx:LatLonQuad>',\n                          '</GroundOverlay>']\n            used.append(img)\n        kml_lines += ['</Document>','</kml>']\n        with zipfile.ZipFile(kmz_path, 'w', compression=zipfile.ZIP_DEFLATED) as zf:\n            zf.writestr(\"doc.kml\", \"\\n\".join(kml_lines))\n            for img in used:\n                zf.write(os.path.join(out_dir, img), f\"{images_dir_in_kmz}/{img}\")\n        return kmz_path\n\n    def write_regionated_kmz(palette_suffix=\"\"):\n        kmz_path = os.path.join(out_dir, f\"{base}_sidescan_regionated{palette_suffix}.kmz\")\n        images_dir_in_kmz = \"files\"\n        # bounds\n        valids=[(lon,lat) for lon,lat in latlons if None not in (lon,lat)]\n        if not valids:\n            return None\n        lons=[a for a,b in valids]; lats=[b for a,b in valids]\n        lat_min, lat_max = min(lats), max(lats)\n        lon_min, lon_max = min(lons), max(lons)\n        # quad folders\n        mid_lat=(lat_min+lat_max)/2.0; mid_lon=(lon_min+lon_max)/2.0\n        quads={\"NW\":(mid_lat,lat_max,lon_min,mid_lon),\"NE\":(mid_lat,lat_max,mid_lon,lon_max),\n               \"SW\":(lat_min,mid_lat,lon_min,mid_lon),\"SE\":(lat_min,mid_lat,mid_lon,lon_max)}\n        buckets={k:[] for k in quads}\n        for idx,(lon,lat) in enumerate(latlons[:-1]):\n            if None in (lon,lat): continue\n            key=(\"N\" if lat>=mid_lat else \"S\")+(\"W\" if lon<mid_lon else \"E\")\n            buckets[key].append(idx)\n        def region_folder(name, lat_min, lat_max, lon_min, lon_max):\n            return [f'<Folder><name>{name}</name>',\n                    '<Region>',\n                    f'<LatLonAltBox><north>{lat_max:.8f}</north><south>{lat_min:.8f}</south><east>{lon_max:.8f}</east><west>{lon_min:.8f}</west></LatLonAltBox>',\n                    '<Lod><minLodPixels>64</minLodPixels><maxLodPixels>-1</maxLodPixels></Lod>',\n                    '</Region>']\n        kml = ['<?xml version=\"1.0\" encoding=\"UTF-8\"?>',\n               '<kml xmlns=\"http://www.opengis.net/kml/2.2\" xmlns:gx=\"http://www.google.com/kml/ext/2.2\">',\n               '<Document>',\n               f'<name>{base} sidescan (regionated{palette_suffix})</name>']\n        for name,(sN,nN,wE,eE) in quads.items():\n            kml += region_folder(name, sN,nN,wE,eE)\n            for idx in buckets[name]:\n                img = strip_paths[idx].replace(\".png\", f\"{palette_suffix}.png\") if palette_suffix else strip_paths[idx]\n                lonA,latA = latlons[idx]\n                lonB,latB = latlons[idx+1] if idx+1 < len(latlons) else latlons[idx]\n                if None in (lonA,latA,lonB,latB): continue\n                quad = segment_quad(latA, lonA, latB, lonB, SWATH_SS)\n                coords_txt = \" \".join([f\"{lon:.8f},{lat:.8f}\" for lon,lat in quad])\n                kml += ['<GroundOverlay>',\n                        f'  <name>{img}</name>',\n                        '  <Icon>',\n                        f'    <href>{images_dir_in_kmz}/{img}</href>',\n                        '  </Icon>',\n                        '  <gx:LatLonQuad>',\n                        f'    <coordinates>{coords_txt}</coordinates>',\n                        '</gx:LatLonQuad>',\n                        '</GroundOverlay>']\n            kml += ['</Folder>']\n        kml += ['</Document>','</kml>']\n        with zipfile.ZipFile(kmz_path, 'w', compression=zipfile.ZIP_DEFLATED) as zf:\n            zf.writestr(\"doc.kml\", \"\\n\".join(kml))\n            for img in strip_paths:\n                img2 = img.replace(\".png\", f\"{palette_suffix}.png\") if palette_suffix else img\n                zf.write(os.path.join(out_dir, img2), f\"{images_dir_in_kmz}/{img2}\")\n        return kmz_path\n\n    # Write KMZ for first palette only (to keep KMZ size reasonable)\n    if MAKE_BUCKETED:\n        write_bucketed_kmz(palette_suffix=f\"_{PALETTES[0]}\")\n    if MAKE_REGION:\n        write_regionated_kmz(palette_suffix=f\"_{PALETTES[0]}\")\n\n    # Optional: MP4\n    if MAKE_VIDEO:\n        mp4 = os.path.join(out_dir, f\"{base}_waterfall.mp4\")\n        writer=None; rows_cache=[]\n        try:\n            for i,png in enumerate(strip_paths):\n                arr=np.array(Image.open(os.path.join(out_dir, png.replace(\".png\", f\"_{PALETTES[0]}.png\"))))\n                if i==0:\n                    Hrow = arr.shape[0]; W = arr.shape[1] if arr.ndim==2 else arr.shape[1]\n                    rows_needed = max(1, VIDEO_H // Hrow)\n                    writer = imageio.get_writer(mp4, fps=VIDEO_FPS, codec='libx264', quality=8)\n                rows_cache.append(arr)\n                if len(rows_cache) > rows_needed: rows_cache.pop(0)\n                if len(rows_cache) == rows_needed:\n                    frame = np.vstack(rows_cache)\n                    writer.append_data(frame)\n                if (i+1) >= VIDEO_MAX: break\n        finally:\n            if writer is not None: writer.close()\n\n    # Return some summary\n    return dict(rows=len(strip_paths), palettes=PALETTES, out_dir=out_dir)\n")
spec = importlib.util.spec_from_file_location("rsd_core_crc", core_path)
core = importlib.util.module_from_spec(spec); sys.modules["rsd_core_crc"]=core; spec.loader.exec_module(core)
print("Core v5.1 loaded.")

Core v5.1 loaded.


In [8]:
# === Hotfix v5.1a: tolerant CRC + resync ===
import logging, struct
import rsd_core_crc as core   # module loaded in the previous cell

# Keep logs informative but not spammy
logging.getLogger().setLevel(logging.WARNING)

# CRC behavior: 'strict' (enforce), 'warn' (try+warn), 'off' (skip CRC)
core.CRC_MODE = "warn"
# How far to scan ahead when resyncing after a parse failure (bytes)
core.MAX_RESYNC_BYTES = 8 * 1024 * 1024

# Wrap varstruct reader to tolerate CRC mismatches in warn/off modes
_orig_read_varstruct = core._read_varstruct
def _read_varstruct_wrap(mm, pos, size, *, validate_crc=True):
    want_crc = validate_crc and (core.CRC_MODE == "strict")
    try:
        return _orig_read_varstruct(mm, pos, size, validate_crc=want_crc)
    except Exception as e:
        if "CRC mismatch" in str(e) and core.CRC_MODE in ("warn", "off"):
            # Re-read without CRC
            return _orig_read_varstruct(mm, pos, size, validate_crc=False)
        raise
core._read_varstruct = _read_varstruct_wrap

# Header resync helper
MAGIC_HDR = struct.pack("<I", core.RSDParser.MAGIC_REC_HDR)
def _resync(mm, start, limit_bytes):
    end = min(len(mm), start + int(limit_bytes))
    idx = mm.find(MAGIC_HDR, start + 1, end)
    return idx if idx != -1 else end

# Safe record iterator with resync on any failure
def parse_records_safe(self, max_records=None):
    mv = memoryview(self.mm)
    pos = self.HEADER_AREA_END
    seen = 0
    while pos + 12 <= self.size:
        rec_start = pos
        try:
            fields, pos_after_hdr = core._read_varstruct(mv, pos, self.size)
            magic = seq = data_size = rec_time_ms = None
            for fn, val in fields:
                if fn == 0 and len(val) == 4: magic = struct.unpack("<I", val)[0]
                elif fn == 2 and len(val) == 4: seq = struct.unpack("<I", val)[0]
                elif fn == 4 and len(val) == 2: data_size = struct.unpack("<H", val)[0]
                elif fn == 5 and len(val) == 4: rec_time_ms = struct.unpack("<I", val)[0]
            if magic != self.MAGIC_REC_HDR:
                raise ValueError("bad magic")

            # Body
            body_start = pos_after_hdr
            channel_id=lat=lon=wt=bd=sc=fsd=lsd=None
            sonar_ofs=sonar_size=None
            if data_size and data_size > 0:
                b_fields, body_end = core._read_varstruct(mv, body_start, self.size)
                def _dec_varint(b):
                    v=0; s=0
                    for bb in b:
                        v |= (bb & 0x7F) << s
                        if not (bb & 0x80): break
                        s += 7
                    return v
                for fn, val in b_fields:
                    if fn == 0: channel_id = int(_dec_varint(val))
                    elif fn == 1: bd  = core.zigzag_to_int32(_dec_varint(val))/1000.0
                    elif fn == 3: fsd = core.zigzag_to_int32(_dec_varint(val))/1000.0
                    elif fn == 4: lsd = core.zigzag_to_int32(_dec_varint(val))/1000.0
                    elif fn == 7 and len(val)==4: sc  = struct.unpack("<I", val)[0]
                    elif fn == 9 and len(val)==4: lat = self.mapunit_to_deg(struct.unpack("<i", val)[0])
                    elif fn ==10 and len(val)==4: lon = self.mapunit_to_deg(struct.unpack("<i", val)[0])
                    elif fn ==11 and len(val)==4: wt  = struct.unpack("<f", val)[0]
                used = body_end - body_start
                sonar_ofs = body_end
                sonar_size = max(0, (data_size or 0) - used)
                pos = body_start + (data_size or 0)
            else:
                pos = body_start

            # Trailer
            if pos + 12 > self.size: break
            tr_magic, chunk_size, _ = struct.unpack("<III", self.mm[pos:pos+12])
            if tr_magic != self.MAGIC_REC_TRL or chunk_size <= 0:
                raise ValueError("bad trailer")

            yield core.RSDRecord(rec_start, channel_id or None, seq or 0, data_size or 0, rec_time_ms or 0,
                                 lat, lon, wt, bd, sc,
                                 sonar_ofs, sonar_size, fsd, lsd)
            pos = rec_start + chunk_size
            seen += 1
            if max_records and seen >= max_records: break

        except Exception:
            pos = _resync(self.mm, rec_start, getattr(core, "MAX_RESYNC_BYTES", 8*1024*1024))
            if pos >= self.size:
                break

# Monkey-patch
core.RSDParser.parse_records = parse_records_safe
print("Hotfix v5.1a installed — CRC_MODE =", core.CRC_MODE)


Hotfix v5.1a installed — CRC_MODE = warn


In [9]:

# @title 🛠️ Configure (GUI) and Pause
import ipywidgets as W, os
from IPython.display import display

w_rsd = W.Text(value="", description="RSD path:")
w_parent = W.Text(value="/content/rsd_runs", description="Parent dir:")
w_run = W.Text(value="run_colab_v5_1", description="Run name:")

w_row = W.IntSlider(value=40, min=10, max=200, step=2, description="Row height")
w_water = W.IntSlider(value=8, min=0, max=64, step=1, description="Water gap px")
w_inv = W.Checkbox(value=True, description="Invert intensity")
w_lo = W.FloatSlider(value=1.0, min=0.0, max=10.0, step=0.1, description="Clip low %")
w_hi = W.FloatSlider(value=99.0, min=50.0, max=100.0, step=0.1, description="Clip high %")
w_gamma = W.FloatSlider(value=1.0, min=0.1, max=3.0, step=0.1, description="Gamma")
w_pal = W.SelectMultiple(options=["amber","blue","grayscale","green","ironbow"], value=("amber","grayscale"), description="Palettes")

w_slant = W.Checkbox(value=True, description="Slant correction")
w_stride = W.IntSlider(value=4, min=1, max=16, step=1, description="Overlay stride")
w_sw_ss = W.FloatText(value=40.0, description="SS swath (m)")

w_bucket = W.Checkbox(value=True, description="Bucketed KMZ")
w_region = W.Checkbox(value=True, description="Regionated KMZ")

w_depthgps = W.Checkbox(value=True, description="Depth+GPS KMLs")
w_gps_step = W.FloatText(value=100.0, description="GPS step m")
w_depth_every = W.IntSlider(value=20, min=1, max=200, step=1, description="Depth every pings")
w_depft = W.FloatText(value=3.0, description="Δdepth > ft")
w_deppct = W.Text(value="", description="% change (opt)")

w_video = W.Checkbox(value=False, description="Make MP4")
w_vfps = W.IntSlider(value=30, min=1, max=60, step=1, description="FPS")
w_vh = W.IntSlider(value=1080, min=200, max=2160, step=10, description="Video height")
w_vmax = W.IntText(value=20000, description="Max frames")

btn_next = W.Button(description="Next ▶", button_style="success")

display(W.VBox([W.HTML("<h3>Set configuration, then click Next</h3>"),
                 w_rsd, w_parent, w_run,
                 W.HBox([w_row,w_water,w_inv]),
                 W.HBox([w_lo,w_hi,w_gamma]),
                 w_pal,
                 W.HBox([w_slant,w_stride,w_sw_ss]),
                 W.HBox([w_bucket,w_region]),
                 W.HTML("<b>Depth & GPS</b>"),
                 W.HBox([w_depthgps,w_gps_step,w_depth_every]),
                 W.HBox([w_depft,w_deppct]),
                 W.HTML("<b>Video</b>"),
                 W.HBox([w_video,w_vfps,w_vh,w_vmax]),
                 btn_next]))

CFG_HOLDER = {}
def on_next(b):
    CFG_HOLDER["cfg"] = dict(
      ROW_HEIGHT_PX=int(w_row.value), WATER_COLUMN_PX=int(w_water.value),
      INVERT=bool(w_inv.value), CLIP_LOW_PCT=float(w_lo.value),
      CLIP_HIGH_PCT=float(w_hi.value), GAMMA=float(w_gamma.value),
      PALETTES=list(w_pal.value),
      APPLY_SLANT=bool(w_slant.value),
      STRIDE=int(w_stride.value), SWATH_M_SS=float(w_sw_ss.value),
      MAKE_BUCKETED_KMZ=bool(w_bucket.value), MAKE_REGIONATED_KMZ=bool(w_region.value),
      MAKE_DEPTH_GPS=bool(w_depthgps.value), GPS_STEP_M=float(w_gps_step.value), DEPTH_EVERY_PINGS=int(w_depth_every.value),
      DEP_FT_THRESHOLD=float(w_depft.value), PCT_CHANGE_MIN=w_deppct.value,
      MAKE_VIDEO=bool(w_video.value), VIDEO_FPS=int(w_vfps.value), VIDEO_HEIGHT=int(w_vh.value), VIDEO_MAX_FRAMES=int(w_vmax.value),
      PREVIEW_MAX_ROWS=2000
    )
    CFG_HOLDER["paths"] = dict(rsd=w_rsd.value.strip(), parent=w_parent.value.strip(), run=w_run.value.strip())
    print("Configuration captured. Now run the next cell.")
btn_next.on_click(on_next)


Configuration captured. Now run the next cell.
Configuration captured. Now run the next cell.


In [11]:

# @title ▶ Run Pipeline (v5.1)
import os
assert "cfg" in CFG_HOLDER and "paths" in CFG_HOLDER, "Please set config in previous cell and click Next."
paths = CFG_HOLDER["paths"]; cfg = CFG_HOLDER["cfg"]
rsd_path = paths["rsd"]; out_parent = paths["parent"]; run = paths["run"]
assert rsd_path and os.path.exists(rsd_path), "RSD path invalid."
out_dir = os.path.join(out_parent, run); os.makedirs(out_dir, exist_ok=True)
summary = core.build_rows_and_assets(rsd_path, out_dir, cfg)
print("Done. Rows:", summary["rows"], "Palettes:", summary["palettes"])
print("Outputs in:", summary["out_dir"])


Done. Rows: 20347 Palettes: ['amber', 'grayscale']
Outputs in: /content/rsd_runs/run_colab_v5_1a


In [12]:

# @title ⬇ Auto-ZIP & Download (Colab)
import shutil, os
from google.colab import files
zip_base = f"{CFG_HOLDER['paths']['run']}_outputs"
zip_path = shutil.make_archive(zip_base, 'zip',
                               root_dir=CFG_HOLDER['paths']['parent'],
                               base_dir=CFG_HOLDER['paths']['run'])
print("ZIP ready at:", zip_path)
files.download(zip_path)


ZIP ready at: /content/run_colab_v5_1a_outputs.zip


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>